In [1]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr

In [2]:
with open("simu6b_data/simu6b_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
df_study = df_study.loc[df_study.session < 100]
df_test = df_test.loc[df_test.session < 100]

In [3]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,list,session
0,120,459,BELT,DIAPER,0,0
1,1315,1260,SKETCH,SEAFOOD,0,0
2,357,723,COMPANION,HOSTESS,0,0
3,1551,1202,VEAL,ROCKET,0,0
4,1081,1066,PLANE,PIGMENT,0,0
...,...,...,...,...,...,...
7195,1019,1143,PASSAGE,PUPIL,5,99
7196,293,1375,CHERRY,STEAM,5,99
7197,871,927,MARKET,MOVIE,5,99
7198,1392,1486,STREAM,TORTOISE,5,99


In [4]:
df_test

,test_itemno1,test_itemno2,test_item,correct_ans,order,rep,test,list,session,test_pos,pair_idx
0,51,-1,ASTRONAUT,1083,2,1,1,0,0,1,8
1,1202,-1,ROCKET,1551,2,1,1,0,0,2,4
2,979,-1,OPERATOR,142,1,1,1,0,0,3,11
3,134,-1,BLADE,105,1,1,1,0,0,4,9
4,1081,-1,PLANE,1066,1,1,1,0,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...
14395,1019,-1,PASSAGE,1143,1,1,2,5,99,20,7196
14396,487,-1,DOORBELL,788,2,1,2,5,99,21,7189
14397,723,-1,HOSTESS,1378,2,1,2,5,99,22,7192
14398,196,-1,BULLY,786,1,1,2,5,99,23,7190


### Run CMR-IA

In [5]:
s_mat = np.load('../../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [6]:
params = cmr.make_default_params()
params.update(learn_while_retrieving=True, nitems_in_accumulator = 96, use_new_context = True)
# params.update(
#     beta_enc = 0.5,
#     beta_rec = 0.1, # beta for retrieved item
#     beta_cue = 0.45, # beta for cue
#     beta_rec_post = 0.99,
#     beta_distract = 0.1,
#     gamma_fc = 0.35,
#     gamma_cf = 0.35,
#     s_fc = 0.1,
#     s_cf = 0.1,
#     c_thresh = 0.4,
#     c_thresh_ass = 0.01,
#     kappa = 0.02,
#     lamb = 0.01,
#     eta = 0.005,
#     phi_s = 1,
#     phi_d = 0.6,
#     nitems_in_accumulator = 96,
#     d_ass = 0.7,
#     learn_while_retrieving = True,
#     omega = 5,
#     alpha = 1
# )
# params

In [7]:
# pso results
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from optimization_utils import make_boundary

_,_,what_to_fit = make_boundary(sim_name='6b')
optim_params = np.loadtxt("../../Modeling/CMR_IA/fitting/outparams_backup/6b_240529_100-100.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
params

{'beta_enc': 0.953602,
 'beta_rec': 0.519641,
 'beta_cue': 0.854426,
 'beta_rec_post': 0.372024,
 'beta_distract': 0.830854,
 'phi_s': 2.17525,
 'phi_d': 0.731406,
 's_cf': 0.051149,
 's_fc': 0.148548,
 'kappa': 0.035835,
 'eta': 0.014504,
 'omega': 6.157514,
 'alpha': 0.957752,
 'c_thresh': 0.301722,
 'c_thresh_itm': 0.5,
 'c_thresh_ass': 0.5,
 'd_ass': 1,
 'lamb': 0.030318,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 96,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'thresh_sigma': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.401852,
 'gamma_cf': 0.541581,
 'use_new_context': True}

In [8]:
df_simu, f_in, f_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat, mode='CR-CR')
# df_simu['test_pos'] = np.tile(np.arange(1,25),6000)
df_simu['test_pos'] = np.tile(np.arange(1,25),600)
df_simu = df_simu.merge(df_test,on=['session','list', 'test_itemno1','test_itemno2', 'test_pos'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

CMR2 Time: 24.2771635055542


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct
0,0,0,51,-1,1083.0,2420.0,0.534844,1,ASTRONAUT,1083,2,1,1,8,True
1,0,0,1202,-1,1551.0,1650.0,0.557797,2,ROCKET,1551,2,1,1,4,True
2,0,0,979,-1,142.0,2440.0,0.539726,3,OPERATOR,142,1,1,1,11,True
3,0,0,134,-1,105.0,2140.0,0.588176,4,BLADE,105,1,1,1,9,True
4,0,0,1081,-1,1066.0,1650.0,0.577245,5,PLANE,1066,1,1,1,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14395,99,5,1019,-1,705.0,1550.0,0.481061,20,PASSAGE,1143,1,1,2,7196,False
14396,99,5,487,-1,788.0,1100.0,0.571058,21,DOORBELL,788,2,1,2,7189,True
14397,99,5,723,-1,705.0,2000.0,0.358500,22,HOSTESS,1378,2,1,2,7192,False
14398,99,5,196,-1,705.0,1070.0,0.608538,23,BULLY,786,1,1,2,7190,False


In [9]:
# get condition and congruence
df_cond = df_simu.groupby(["pair_idx","test"])['order'].mean().to_frame(name='corr_rate').reset_index()
df_cond = df_cond.pivot_table(index='pair_idx',columns='test',values='corr_rate').reset_index()
df_cond.columns = ['pair_idx','test1','test2']

def cond(x):
    test1 = x['test1']
    test2 = x['test2']
    if test1 == 1 and test2 == 1:
        return 'F-F'
    elif test1 == 1 and test2 == 2:
        return 'F-B'
    elif test1 == 2 and test2 == 1:
        return 'B-F'
    elif test1 == 2 and test2 == 2:
        return 'B-B'

df_cond['cond'] = df_cond.apply(lambda x:cond(x),axis=1)
df_cond['cong'] = df_cond.apply(lambda x: 'Identical' if x['cond'] == 'F-F' or x['cond'] == 'B-B' else 'Reversed',axis=1)
df_cond

,pair_idx,test1,test2,cond,cong
0,1,1.0,1.0,F-F,Identical
1,2,2.0,1.0,B-F,Reversed
2,3,1.0,1.0,F-F,Identical
3,4,2.0,1.0,B-F,Reversed
4,5,1.0,2.0,F-B,Reversed
...,...,...,...,...,...
7195,7196,2.0,1.0,B-F,Reversed
7196,7197,1.0,1.0,F-F,Identical
7197,7198,2.0,1.0,B-F,Reversed
7198,7199,1.0,2.0,F-B,Reversed


In [10]:
# merge condition and congruence
pairidx2cond = df_cond.loc[:,['pair_idx','cond']].set_index("pair_idx").to_dict()['cond']
pairidx2cong = df_cond.loc[:,['pair_idx','cong']].set_index("pair_idx").to_dict()['cong']
df_simu['cond'] = df_simu.apply(lambda x:pairidx2cond[x['pair_idx']],axis=1)
df_simu['cong'] = df_simu.apply(lambda x:pairidx2cong[x['pair_idx']],axis=1)
df_simu.head(24)

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong
0,0,0,51,-1,1083.0,2420.0,0.534844,1,ASTRONAUT,1083,2,1,1,8,True,B-B,Identical
1,0,0,1202,-1,1551.0,1650.0,0.557797,2,ROCKET,1551,2,1,1,4,True,B-F,Reversed
2,0,0,979,-1,142.0,2440.0,0.539726,3,OPERATOR,142,1,1,1,11,True,F-F,Identical
3,0,0,134,-1,105.0,2140.0,0.588176,4,BLADE,105,1,1,1,9,True,F-B,Reversed
4,0,0,1081,-1,1066.0,1650.0,0.577245,5,PLANE,1066,1,1,1,5,True,F-B,Reversed
5,0,0,357,-1,723.0,1450.0,0.585341,6,COMPANION,723,1,1,1,3,True,F-F,Identical
6,0,0,831,-1,1435.0,2860.0,0.543778,7,LITERATURE,1435,2,1,1,7,True,B-B,Identical
7,0,0,1188,-1,641.0,2020.0,0.531187,8,RIB,641,2,1,1,12,True,B-F,Reversed
8,0,0,1260,-1,1315.0,1030.0,0.565651,9,SEAFOOD,1315,2,1,1,2,True,B-F,Reversed
9,0,0,12,-1,226.0,3640.0,0.581178,10,AIRPLANE,226,1,1,1,10,True,F-B,Reversed


In [11]:
# get f_in
sessions = np.unique(df_simu.session.to_numpy())
tmp_corr_fin = []
tmp_omean_fin = []
tmp_omax_fin = []

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    tmp = np.concatenate((tmp1, tmp2))
    tmp = np.sort(tmp)

    df_tmp1 = df_simu.loc[df_simu.session == sess]
    tmp_test = df_tmp1.test_itemno1
    tmp_corr = df_tmp1.correct_ans
    # tmp_test = df_simu.loc[df_simu.session == sess, 'test_itemno']
    # tmp_corr = df_simu.loc[df_simu.session == sess, 'correct_ans']
    testid = np.searchsorted(tmp, tmp_test)
    corrid = np.searchsorted(tmp, tmp_corr)

    fin_sess = f_dif[sess]
    # tmp_corr_fin = []
    # tmp_omean_fin = []
    # tmp_omax_fin = []
    for i in range(len(testid)):
        tmp_corr_fin.append(fin_sess[i][corrid[i]])
        tmp_omean_fin.append(np.mean(np.delete(fin_sess[i], corrid[i])))
        tmp_omax_fin.append(np.delete(fin_sess[i], corrid[i]).max())
    # df_simu.loc[df_simu.session == sess, 'corr_fin']= tmp_corr_fin
    # df_simu.loc[df_simu.session == sess, 'omean_fin'] = tmp_omean_fin
    # df_simu.loc[df_simu.session == sess, 'omax_fin'] = tmp_omax_fin

df_simu['corr_fin']= tmp_corr_fin
df_simu['omean_fin'] = tmp_omean_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,0,51,-1,1083.0,2420.0,0.534844,1,ASTRONAUT,1083,2,1,1,8,True,B-B,Identical,-0.442592,-0.993926,-0.709784
1,0,0,1202,-1,1551.0,1650.0,0.557797,2,ROCKET,1551,2,1,1,4,True,B-F,Reversed,-0.318100,-1.065843,-0.739677
2,0,0,979,-1,142.0,2440.0,0.539726,3,OPERATOR,142,1,1,1,11,True,F-F,Identical,-0.435481,-1.149513,-0.768745
3,0,0,134,-1,105.0,2140.0,0.588176,4,BLADE,105,1,1,1,9,True,F-B,Reversed,-0.407076,-1.212024,-0.663599
4,0,0,1081,-1,1066.0,1650.0,0.577245,5,PLANE,1066,1,1,1,5,True,F-B,Reversed,-0.357837,-1.282700,-0.702991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14395,99,5,1019,-1,705.0,1550.0,0.481061,20,PASSAGE,1143,1,1,2,7196,False,B-F,Reversed,-0.098031,-1.043995,0.014541
14396,99,5,487,-1,788.0,1100.0,0.571058,21,DOORBELL,788,2,1,2,7189,True,B-B,Identical,0.587214,-0.910500,0.277996
14397,99,5,723,-1,705.0,2000.0,0.358500,22,HOSTESS,1378,2,1,2,7192,False,F-B,Reversed,-0.299069,-1.156219,-0.053291
14398,99,5,196,-1,705.0,1070.0,0.608538,23,BULLY,786,1,1,2,7190,False,F-F,Identical,0.103318,-1.020121,0.180667


In [12]:
df_fin = df_simu.groupby(['cong','test'])[['corr_fin','omean_fin','omax_fin']].mean().reset_index()
df_fin['dif'] = df_fin['corr_fin'] - df_fin['omax_fin']
df_fin

,cong,test,corr_fin,omean_fin,omax_fin,dif
0,Identical,1,-0.244425,-1.142348,0.138236,-0.382661
1,Identical,2,-0.064439,-1.091566,0.707475,-0.771914
2,Reversed,1,-0.246102,-1.141822,0.136161,-0.382263
3,Reversed,2,0.106289,-1.121742,0.395191,-0.288901


### Analysis

In [13]:
# anal_perform used by pso
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from object_funcs import anal_perform_6b as anal_perform

In [14]:
# def anal_perform(df_simu):

#     # get pair
#     df_pair = pd.pivot_table(df_simu,index='pair_idx',columns='test', values= 'correct')
#     df_pair.columns = ['test1','test2']
#     test2_rsp = pd.Categorical(df_pair.test2, categories=[1,0])
#     test1_rsp = pd.Categorical(df_pair.test1, categories=[1,0])
#     df_tab = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize=False, dropna=False)
#     df_tab_norm = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize='all', dropna=False)
#     t1_t2 = df_tab_norm[1][1] # 1, 2
#     t1_f2 = df_tab_norm[1][0]
#     f1_t2 = df_tab_norm[0][1]
#     f1_f2 = df_tab_norm[0][0]
#     # print(df_tab)
#     # print(df_tab_norm)
#     # print(t1_t2, t1_f2, f1_t2, f1_f2)

#     # compute" Q
#     def Yule_Q(A, B, C, D):
#         return (A * D - B * C) / (A * D + B * C)
#     q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)  # add 0.5
#     # print("Q: ", q)

#     return t1_t2, t1_f2, f1_t2, f1_f2, q

In [15]:
subjects = np.unique(df_simu.session)
inde_stats = []
reve_stats = []
for subj in subjects:
    df_subj_inde = df_simu.query(f"session == {subj} and cong == 'Identical'").copy()
    inde_stats.append(list(anal_perform(df_subj_inde)))

    df_subj_reve = df_simu.query(f"session == {subj} and cong == 'Reversed'").copy()
    reve_stats.append(list(anal_perform(df_subj_reve)))

In [16]:
np.array(inde_stats)

array([[0.25      , 0.05555556, 0.        , 0.69444444, 0.98973306],
       [0.33333333, 0.        , 0.02777778, 0.63888889, 0.99490662],
       [0.25      , 0.02777778, 0.08333333, 0.63888889, 0.95404814],
       [0.27777778, 0.05555556, 0.        , 0.66666667, 0.99032882],
       [0.30555556, 0.        , 0.02777778, 0.66666667, 0.99469027],
       [0.27777778, 0.05555556, 0.        , 0.66666667, 0.99032882],
       [0.38888889, 0.02777778, 0.02777778, 0.55555556, 0.98497496],
       [0.36111111, 0.        , 0.        , 0.63888889, 0.9984252 ],
       [0.30555556, 0.02777778, 0.        , 0.66666667, 0.99469027],
       [0.27777778, 0.02777778, 0.        , 0.69444444, 0.99441341],
       [0.27777778, 0.02777778, 0.02777778, 0.66666667, 0.98265896],
       [0.30555556, 0.05555556, 0.        , 0.63888889, 0.9907919 ],
       [0.47222222, 0.05555556, 0.02777778, 0.44444444, 0.97435897],
       [0.19444444, 0.        , 0.        , 0.80555556, 0.99774266],
       [0.36111111, 0.        , 0.

In [17]:
np.array(reve_stats)

array([[0.27777778, 0.        , 0.16666667, 0.55555556, 0.97025172],
       [0.36111111, 0.08333333, 0.08333333, 0.47222222, 0.90140845],
       [0.25      , 0.02777778, 0.08333333, 0.63888889, 0.95404814],
       [0.25      , 0.        , 0.05555556, 0.69444444, 0.98973306],
       [0.25      , 0.        , 0.11111111, 0.63888889, 0.98004435],
       [0.30555556, 0.        , 0.19444444, 0.5       , 0.96535797],
       [0.33333333, 0.02777778, 0.19444444, 0.44444444, 0.89655172],
       [0.36111111, 0.        , 0.05555556, 0.58333333, 0.99142367],
       [0.36111111, 0.        , 0.11111111, 0.52777778, 0.98305085],
       [0.27777778, 0.        , 0.13888889, 0.58333333, 0.97592998],
       [0.19444444, 0.        , 0.08333333, 0.72222222, 0.98254364],
       [0.38888889, 0.05555556, 0.08333333, 0.47222222, 0.93333333],
       [0.58333333, 0.        , 0.13888889, 0.27777778, 0.97592998],
       [0.27777778, 0.02777778, 0.11111111, 0.58333333, 0.94193548],
       [0.19444444, 0.02777778, 0.

In [18]:
np.mean(inde_stats,axis=0)

array([0.30388889, 0.02444444, 0.01944444, 0.65222222, 0.98672516])

In [19]:
np.mean(reve_stats,axis=0)

array([0.30583333, 0.02611111, 0.11583333, 0.55222222, 0.94484125])

In [20]:
inde_stats_mean = np.mean(inde_stats,axis=0)
reve_stats_mean = np.mean(reve_stats,axis=0)
inde_ground_truth = np.array([0.319, 0.006, 0.012, 0.663, 0.94])
reve_ground_truth = np.array([0.293, 0.049, 0.122, 0.537, 0.96])
err = (np.mean(np.power(inde_stats_mean-inde_ground_truth,2)) + np.mean(np.power(reve_stats_mean-reve_ground_truth,2)))/2
err

0.0004111491500103194

### Whole

In [21]:
dfgp_test_order = df_simu.groupby(['rep','test','order']).correct.mean().to_frame(name='corr_rate').reset_index()
dfgp_test_order

,rep,test,order,corr_rate
0,1,1,1,0.330556
1,1,1,2,0.329722
2,1,2,1,0.364722
3,1,2,2,0.380278


In [22]:
df_simu.groupby(['cong','test']).correct.mean()

cong       test
Identical  1       0.328333
           2       0.323333
Reversed   1       0.331944
           2       0.421667
Name: correct, dtype: float64

In [23]:
def contingency_tab(x):
    tmp_x = pd.pivot_table(x,index='pair_idx',columns='test', values= 'correct')
    test1 = pd.Categorical(tmp_x[1], categories=[1, 0])
    test2 = pd.Categorical(tmp_x[2], categories=[1, 0])
    return pd.crosstab(index=test2,columns=test1, normalize='all')

In [24]:
dfgp_rep_cong = df_simu.groupby(['cong']).apply(contingency_tab)
dfgp_rep_cong

col_0                   1         0
cong      row_0                    
Identical 1      0.303889  0.019444
          0      0.024444  0.652222
Reversed  1      0.305833  0.115833
          0      0.026111  0.552222

In [25]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)

In [26]:
tab_identical = dfgp_rep_cong.loc['Identical']
Yule_Q(tab_identical[1][1],tab_identical[0][1],tab_identical[1][0],tab_identical[0][0])

0.9952152961389925

In [27]:
tab_reversed = dfgp_rep_cong.loc['Reversed']
Yule_Q(tab_reversed[1][1],tab_reversed[0][1],tab_reversed[1][0],tab_reversed[0][0])

0.9648130643549825

In [28]:
Yule_Q(0.319, 0.012, 0.006, 0.663)

0.9993193709853524

In [29]:
Yule_Q(0.293, 0.122, 0.049, 0.537)

0.9267935757627709